In [1]:
import requests
from bs4 import BeautifulSoup
from retry import retry
import urllib
import time
import numpy as np
import pandas as pd
from datetime import date

from sqlalchemy import create_engine
from sqlalchemy import text

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from joblib import dump
from joblib import load

In [2]:
# 複数ページの情報をまとめて取得
data_samples = []

# スクレイピングするページ数
max_page = 100
# SUUMOを桜山のみ指定して検索して出力した画面のurl(ページ数フォーマットが必要)
base_url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=050&bs=040&ra=023&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&ek=323016210&rn=3230'

# リクエストがうまく行かないパターンを回避するためのやり直し
@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

# 処理時間を測りたい
start = time.time()
times = []


for page in range(1, max_page + 1):
    # ページネーション用のクエリパラメータを追加
    url = f'{base_url}&page={page}'
    soup = load_page(url)
    before = time.time()
    # 物件情報リストを指定
    mother = soup.find_all(class_='cassetteitem')
        
    # 物件ごとの処理
    for child in mother:

        # 建物情報
        data_home = []
        # カテゴリ
        data_home.append(child.find(class_='ui-pct ui-pct--util1').text)
        # 建物名
        data_home.append(child.find(class_='cassetteitem_content-title').text)
        # 住所
        data_home.append(child.find(class_='cassetteitem_detail-col1').text)
        # 最寄り駅のアクセス
        children = child.find(class_='cassetteitem_detail-col2')
        for id,grandchild in enumerate(children.find_all(class_='cassetteitem_detail-text')):
            data_home.append(grandchild.text)
        # 築年数と階数
        children = child.find(class_='cassetteitem_detail-col3')
        for grandchild in children.find_all('div'):
            data_home.append(grandchild.text)

        # 部屋情報
        rooms = child.find(class_='cassetteitem_other')
        for room in rooms.find_all(class_='js-cassette_link'):
            data_room = []
            
            # 部屋情報が入っている表を探索
            for id_, grandchild in enumerate(room.find_all('td')):
                # 階
                if id_ == 2:
                    data_room.append(grandchild.text.strip())
                # 家賃と管理費
                elif id_ == 3:
                    data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text)
                # 敷金と礼金
                elif id_ == 4:
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
                # 間取りと面積
                elif id_ == 5:
                    data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                    data_room.append(grandchild.find(class_='cassetteitem_menseki').text)
                # url
                elif id_ == 8:
                    get_url = grandchild.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
                    abs_url = urllib.parse.urljoin(url,get_url)
                    data_room.append(abs_url)
            # 物件情報と部屋情報をくっつける
            data_sample = data_home + data_room
            data_samples.append(data_sample)
    
    # 1アクセスごとに1秒休む
    time.sleep(1)
    
    # 進捗確認
    # このページの作業時間を表示
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'{page}ページ目：{running_time}秒')
    # 取得した件数
    print(f'総取得件数：{len(data_samples)}')
    # 作業進捗
    complete_ratio = round(page/max_page*100,3)
    print(f'完了：{complete_ratio}%')
    # 作業の残り時間目安を表示
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間：{hour}時間{minute}分{second}秒\n')

# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：',running_all)

1ページ目：1.2294299602508545秒
総取得件数：108
完了：1.0%
残り時間：0時間2分1秒

2ページ目：1.169057846069336秒
総取得件数：195
完了：2.0%
残り時間：0時間1分57秒

3ページ目：1.1422007083892822秒
総取得件数：293
完了：3.0%
残り時間：0時間1分54秒

4ページ目：1.1156432628631592秒
総取得件数：358
完了：4.0%
残り時間：0時間1分51秒

5ページ目：1.1586649417877197秒
総取得件数：420
完了：5.0%
残り時間：0時間1分50秒

6ページ目：1.1045351028442383秒
総取得件数：480
完了：6.0%
残り時間：0時間1分48秒

7ページ目：1.1081740856170654秒
総取得件数：538
完了：7.0%
残り時間：0時間1分46秒

8ページ目：1.1172077655792236秒
総取得件数：614
完了：8.0%
残り時間：0時間1分45秒

9ページ目：1.1291100978851318秒
総取得件数：702
完了：9.0%
残り時間：0時間1分43秒

10ページ目：1.1236419677734375秒
総取得件数：785
完了：10.0%
残り時間：0時間1分42秒

11ページ目：1.0874030590057373秒
総取得件数：830
完了：11.0%
残り時間：0時間1分41秒

12ページ目：1.1048202514648438秒
総取得件数：894
完了：12.0%
残り時間：0時間1分39秒

13ページ目：1.1100518703460693秒
総取得件数：969
完了：13.0%
残り時間：0時間1分38秒

14ページ目：1.121535062789917秒
総取得件数：1027
完了：14.0%
残り時間：0時間1分37秒

15ページ目：1.0880708694458008秒
総取得件数：1077
完了：15.0%
残り時間：0時間1分35秒

16ページ目：1.0914099216461182秒
総取得件数：1130
完了：16.0%
残り時間：0時間1分34秒

17ページ目：1.1414389610290527秒
総取得件数：1232
完了：1

In [3]:
# data_samplesをDataFrameに変換
columns = ['property_type', 'building_name', 'address', 'access_1', 'access_2', 'access_3',
    'age', 'building_floors', 'room_floor', 'rent', 'management_fee',
    'deposit', 'gratuity', 'layout', 'area', 'url']
df = pd.DataFrame(data_samples, columns=columns)

# 現在の日付を掲載日として入力
current_date = date.today()
df['posting_date'] = current_date

In [4]:
# 重複行を削除
df_unique = df.drop_duplicates()

df_unique

,property_type,building_name,address,access_1,access_2,access_3,age,building_floors,room_floor,rent,management_fee,deposit,gratuity,layout,area,url,posting_date
0,賃貸アパート,ネオ プリムローズ,愛知県名古屋市瑞穂区川澄町４,地下鉄桜通線/桜山駅 歩6分,地下鉄桜通線/瑞穂区役所駅 歩11分,地下鉄桜通線/瑞穂運動場西駅 歩21分,築17年,2階建,1階,6.05万円,3300円,-,-,1K,30.01m2,https://suumo.jp/chintai/jnc_000088658772/?bc=...,2024-02-23
1,賃貸マンション,スリーアイランドタワー桜山,愛知県名古屋市昭和区広見町４,地下鉄桜通線/桜山駅 歩5分,地下鉄鶴舞線/荒畑駅 歩19分,地下鉄桜通線/瑞穂区役所駅 歩18分,築1年,9階建,3階,7.1万円,7000円,7.1万円,7.1万円,1K,27.72m2,https://suumo.jp/chintai/jnc_000087842850/?bc=...,2024-02-23
2,賃貸マンション,スリーアイランドタワー桜山,愛知県名古屋市昭和区広見町４,地下鉄桜通線/桜山駅 歩5分,地下鉄鶴舞線/荒畑駅 歩19分,地下鉄桜通線/瑞穂区役所駅 歩18分,築1年,9階建,5階,7.3万円,7000円,7.3万円,7.3万円,1K,27.72m2,https://suumo.jp/chintai/jnc_000087842852/?bc=...,2024-02-23
3,賃貸マンション,スリーアイランドタワー桜山,愛知県名古屋市昭和区広見町４,地下鉄桜通線/桜山駅 歩5分,地下鉄鶴舞線/荒畑駅 歩19分,地下鉄桜通線/瑞穂区役所駅 歩18分,築1年,9階建,6階,7.4万円,7000円,7.4万円,7.4万円,1K,27.72m2,https://suumo.jp/chintai/jnc_000087842853/?bc=...,2024-02-23
4,賃貸マンション,スリーアイランドタワー桜山,愛知県名古屋市昭和区広見町４,地下鉄桜通線/桜山駅 歩5分,地下鉄鶴舞線/荒畑駅 歩19分,地下鉄桜通線/瑞穂区役所駅 歩18分,築1年,9階建,8階,7.6万円,7000円,7.6万円,7.6万円,1K,27.72m2,https://suumo.jp/chintai/jnc_000087842856/?bc=...,2024-02-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2079,賃貸アパート,プチ長池ハイツ,愛知県名古屋市昭和区長池町３,地下鉄鶴舞線/川名駅 歩12分,地下鉄桜通線/桜山駅 歩13分,地下鉄鶴舞線/御器所駅 歩15分,築42年,2階建,1階,2.6万円,3000円,4万円,-,1K,21m2,https://suumo.jp/chintai/jnc_000015704417/?bc=...,2024-02-23
2080,賃貸マンション,地下鉄鶴舞線 御器所駅 2階建 築51年,愛知県名古屋市昭和区長戸町１,地下鉄鶴舞線/御器所駅 歩12分,地下鉄桜通線/桜山駅 歩13分,地下鉄鶴舞線/川名駅 歩14分,築51年,2階建,2階,6.05万円,3300円,-,-,ワンルーム,39.66m2,https://suumo.jp/chintai/jnc_000088547081/?bc=...,2024-02-23
2081,賃貸一戸建て,長戸町4丁目貸家,愛知県名古屋市昭和区長戸町４,地下鉄鶴舞線/川名駅 歩10分,地下鉄桜通線/桜山駅 歩12分,地下鉄桜通線/御器所駅 歩14分,築87年,2階建,1-2階,12万円,-,12万円,12万円,3LDK,69.99m2,https://suumo.jp/chintai/jnc_000085151800/?bc=...,2024-02-23
2082,賃貸アパート,ふじハイツ,愛知県名古屋市昭和区円上町,地下鉄鶴舞線/荒畑駅 歩15分,地下鉄桜通線/桜山駅 歩18分,地下鉄鶴舞線/鶴舞駅 歩21分,築44年,2階建,2階,3.5万円,2000円,3.5万円,-,1K,21.53m2,https://suumo.jp/chintai/jnc_000088444956/?bc=...,2024-02-23


# データの保存

In [5]:
df_unique.to_csv('scraped_data.csv')

# データベース関連の処理

In [6]:
# CSVファイルを読み込む
df = pd.read_csv('scraped_data.csv', index_col=0)

In [7]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

db_user = config['database']['user']
db_password = config['database']['password']
db_host = config['database']['host']
db_name = config['database']['name']

In [8]:
# PostgreSQLデータベースへの接続情報
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}/{db_name}')

# dbを読み込み
db_df = pd.read_sql_table('rental_properties', engine)

In [9]:
# 新規レコードの挿入
## スクレイピングしたデータ（df）にあって、データベース（db_df）にないレコードを特定
new_records = df.merge(db_df, on=['url'], how='left', indicator=True).loc[lambda x : x['_merge']=='left_only']

## 不要なカラムを削除
new_records = new_records.drop(columns=['_merge'])

## 新規レコードに掲載日を設定
#new_records['posting_date'] = current_date

# カラム名を修正するためのrename操作
new_records.rename(columns={
    'property_type_x': 'property_type',
    'building_name_x': 'building_name',
    'address_x': 'address',
    'access_1_x': 'access_1',
    'access_2_x': 'access_2',
    'access_3_x': 'access_3',
    'age_x': 'age',
    'building_floors_x': 'building_floors',
    'room_floor_x': 'room_floor',
    'rent_x': 'rent',
    'management_fee_x': 'management_fee',
    'deposit_x': 'deposit',
    'gratuity_x': 'gratuity',
    'layout_x': 'layout',
    'area_x': 'area',
    'url_x': 'url',
    'posting_date_x': 'posting_date'
}, inplace=True)

# 例: 不要なカラムを削除し、必要なカラムのみを選択
cleaned_df = new_records[['property_type', 'building_name', 'address', 'access_1', 'access_2', 'access_3', 'age', 'building_floors', 'room_floor', 'rent', 'management_fee', 'deposit', 'gratuity', 'layout', 'area', 'url', 'posting_date']].copy()

## 新規レコードをデータベースに挿入
cleaned_df.to_sql('rental_properties', engine, if_exists='append', index=False)

In [10]:
# 削除されたレコードの特定と更新
## データベースにあって、スクレイピングしたデータにないレコードを特定
removed_records = db_df.merge(df, on=['url'], how='left', indicator=True).loc[lambda x : x['_merge'] == 'left_only']

# removed_dateがNULL（つまり、未設定）のレコードに対してのみ更新を行う
for index, row in removed_records.iterrows():
    # removed_dateが未設定かどうかを確認するためのクエリ
    check_query = text("SELECT removed_date FROM rental_properties WHERE url = :url AND removed_date IS NULL")
    result = engine.execute(check_query, url=row['url']).fetchone()
    
    # removed_dateが未設定の場合のみ、removed_dateを更新する
    if result:
        update_query = text("UPDATE rental_properties SET removed_date = :current_date WHERE url = :url AND removed_date IS NULL")
        engine.execute(update_query, current_date=current_date, url=row['url'])

# 機械学習モデルによる予測値の入力

In [11]:
# dbを読み込み
df = pd.read_sql_table('rental_properties', engine)
ml_df = pd.read_sql_table('rental_properties', engine)

## 前処理

In [12]:
# 共益費management_feeと敷金(deposit)と礼金(gratuity)が「-」の場合は0とする
ml_df['management_fee'] = ml_df['management_fee'].replace('-', '0円')
ml_df['deposit'] = ml_df['deposit'].replace('-', '0万円')
ml_df['gratuity'] = ml_df['gratuity'].replace('-', '0万円')

# 金額関連の列を数値に変換する関数
def yen_to_float(yen_str):
    return float(yen_str.replace('万円', '')) * 10000

# rent, management_fee, deposit, gratuity を数値に変換
ml_df['rent'] = ml_df['rent'].apply(yen_to_float)
ml_df['management_fee'] = ml_df['management_fee'].str.replace('円', '').astype(float)
ml_df['deposit'] = ml_df['deposit'].apply(yen_to_float)
ml_df['gratuity'] = ml_df['gratuity'].apply(yen_to_float)

# 面積を数値に変換
ml_df['area'] = ml_df['area'].str.replace('m2', '').astype(float)

# 実質負担額を計算（rent + management_fee + (deposit + gratuity) / 48）
ml_df['actual_cost'] = ml_df['rent'] + ml_df['management_fee'] + (ml_df['deposit'] + ml_df['gratuity']) / 48

In [13]:
# 桜山駅からの最短時間を取得する関数
def get_min_time_to_sakurayama(access_str):
    if pd.isnull(access_str):
        return np.nan
    times = [int(s.split('歩')[1].replace('分', '')) for s in access_str.split('/') if '桜山駅' in s]
    if times:
        return min(times)
    else:
        return np.nan

# access_1, access_2, access_3から桜山駅までの最短時間を計算
ml_df['min_time_to_sakurayama'] = ml_df[['access_1', 'access_2', 'access_3']].apply(
    lambda x: min(
        filter(pd.notnull, [get_min_time_to_sakurayama(x['access_1']), get_min_time_to_sakurayama(x['access_2']), get_min_time_to_sakurayama(x['access_3'])]),
        default=np.nan
    ),
    axis=1
)

In [14]:
# '築新'または'新'を'0'年として扱うための処理を追加
ml_df['age'] = ml_df['age'].str.replace('築新', '0').str.replace('新', '0').str.replace('築', '').str.replace('年', '')


# 空の値を0に置き換え
ml_df['age'] = ml_df['age'].replace('', '0')

# 整数型に変換
ml_df['age'] = ml_df['age'].astype(int)

# 建物の階数と部屋の階についても同様に処理
ml_df['building_floors'] = ml_df['building_floors'].str.extract('(\d+)')[0].fillna('0').astype(int)
ml_df['room_floor'] = ml_df['room_floor'].str.extract('(\d+)')[0].fillna('0').astype(int)

In [15]:
# 不要なカラムを削除
drop_cols = ['posting_date', 'removed_date', 'building_name', 'access_1', 'access_2', 'access_3', 'posting_date']
ml_df.drop(drop_cols, axis=1, inplace=True)

In [16]:
ml_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2783 entries, 0 to 2782
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      2783 non-null   int64  
 1   property_type           2783 non-null   object 
 2   address                 2783 non-null   object 
 3   age                     2783 non-null   int64  
 4   building_floors         2783 non-null   int64  
 5   room_floor              2783 non-null   int64  
 6   rent                    2783 non-null   float64
 7   management_fee          2783 non-null   float64
 8   deposit                 2783 non-null   float64
 9   gratuity                2783 non-null   float64
 10  layout                  2783 non-null   object 
 11  area                    2783 non-null   float64
 12  url                     2783 non-null   object 
 13  predicted_cost_gb       2037 non-null   float64
 14  predicted_cost_rf       2037 non-null   

In [17]:
# エンコーディングするカラム
categorical_cols = ['property_type', 'address', 'layout']

# One-hotエンコーディング
ml_df = pd.get_dummies(ml_df, columns=categorical_cols)

## モデルでの予測

In [18]:
ml_df.columns = ml_df.columns.astype(str)

# 特徴量とターゲット変数に分割
X_new = ml_df.drop(['url','actual_cost','predicted_cost_rf','predicted_cost_gb'], axis=1)  # actual_cost以外のすべてのカラムを特徴量として使用
y_new = ml_df['actual_cost']  # actual_costをターゲット変数として使用

In [19]:
# 保存されたモデルをロードする
random_forest_model = load('random_forest_model.joblib')
gradient_boosting_model = load('gradient_boosting_model.joblib')

# ランダムフォレストモデルで予測する
predictions_rf = random_forest_model.predict(X_new)

# グラディエントブースティングモデルで予測する
predictions_gb = gradient_boosting_model.predict(X_new)

# 予測結果をml_dfに追加
ml_df['predicted_cost_rf'] = predictions_rf
ml_df['predicted_cost_gb'] = predictions_gb

In [20]:
from sqlalchemy.sql import text

# データベースにデータを格納する（例: 'rental_properties' テーブルを更新）
for index, row in ml_df.iterrows():
    # ここでは、URLを一意の識別子と仮定して、そのURLに対応するレコードを更新します。
    update_query = text("""
    UPDATE rental_properties
    SET predicted_cost_rf = :predicted_cost_rf, predicted_cost_gb = :predicted_cost_gb
    WHERE url = :url
    """)
    engine.execute(update_query, {'predicted_cost_rf': row['predicted_cost_rf'], 'predicted_cost_gb': row['predicted_cost_gb'], 'url': row['url']})
